In [ ]:
# !python /home/vs428/Documents/deep-ed-diags/simple_nn_runner.py

# Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Path hack.
import sys
import os

sys.path.append(os.path.abspath(".."))

In [3]:
import csv
import ast
from datetime import date, datetime, time, timedelta
from datetime import datetime
import pickle as pkl
import time

from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.utils.class_weight import compute_class_weight

import numpy as np
import pandas as pd
import sklearn
from joblib import dump, load

import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

from models.simple_nn import *

In [4]:
def column_index(df, query_cols):
    cols = df.columns.values
    sidx = np.argsort(cols)
    return sidx[np.searchsorted(cols, query_cols, sorter=sidx)]


def flatten(t):
    return [item for sublist in t for item in sublist]

# Dataset

In [5]:
data = pd.read_pickle(
    #     f"/gpfs/milgram/project/rtaylor/shared/ABDPain_EarlyDiags/unq_pt_enc_clean_multilabel_nomis_dvemb.pkl",
    f"/gpfs/milgram/project/rtaylor/shared/ABDPain_EarlyDiags/unq_pt_enc_clean_multilabel_nomismatches.pkl",
    #                   nrows=50000
)

# Read in Other Files

In [9]:
labs = pd.read_csv("/home/vs428/project/Data/abd_pain_cohort/abd_labs_cleaned.csv")
vitals = pd.read_csv("/home/vs428/project/Data/abd_pain_cohort/abd_vitals.csv")
diags = pd.read_csv("/home/vs428/project/Data/abd_pain_cohort/abd_ed_diagnoses.csv")
meds = pd.read_csv("/home/vs428/project/Data/abd_pain_cohort/abd_outpt_med.csv")
pmh = pd.read_csv("/home/vs428/project/Data/abd_pain_cohort/abd_diag.csv")

In [10]:
demos = pd.read_csv("/home/vs428/project/Data/abd_pain_cohort/abd_demos.csv")

# Investigate Patients

In [43]:
data.sample(100)

,PAT_ENC_CSN_ID,person_id,visit_occurrence_id,age,EdDisposition_AMA,EdDisposition_Admit,EdDisposition_Discharge,EdDisposition_Eloped,EdDisposition_Expired,EdDisposition_LWBS after Triage,...,last_BP_Diastolic,ed_SpO2,ed_Temp,ed_Patient Acuity,ed_Pulse,ed_Pain Score,ed_Resp,ed_BP_Systolic,ed_BP_Diastolic,multilabel
6725,99735863,13362040,20581757,35.0,NaN,NaN,1.0,NaN,NaN,NaN,...,96.0,98.0,97.8,3.0,69.0,8.0,18.0,117.0,81.0,[Acute cystitis]
156801,244932576,38355316,234297978,26.0,NaN,NaN,1.0,NaN,NaN,NaN,...,120.0,99.0,98.8,3.0,95.0,8.0,16.0,103.0,67.0,"[Abdominal Pain, general]"
44856,143256103,19927233,62657155,18.0,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,100.0,97.9,3.0,78.0,9.0,26.0,105.0,68.0,[Mesenteric lymphadenitis]
135223,221611436,12204156,206110320,67.0,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,96.0,98.2,3.0,88.0,NaN,16.0,143.0,96.0,"[Appendicitis, Bowel perforation, Peritonitis]"
6611,99618059,13633928,20475040,12.0,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[Abdominal Pain, general]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8514,101570163,12792611,22357435,28.0,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,100.0,98.0,3.0,90.0,5.0,20.0,118.0,82.0,[Gastritis]
35313,131530131,39061670,51144236,39.0,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,97.0,96.5,3.0,102.0,3.0,18.0,108.0,64.0,[Colitis]
157672,245866942,13550914,235334485,34.0,NaN,1.0,NaN,NaN,NaN,NaN,...,87.0,100.0,98.2,3.0,69.0,10.0,18.0,164.0,105.0,[Cholangitis]
145189,230987959,11433183,217256501,21.0,NaN,NaN,1.0,NaN,NaN,NaN,...,72.0,99.0,97.9,3.0,107.0,8.0,16.0,127.0,81.0,[Ovarian cyst]


In [44]:
PAT_ENC_CSN_ID = 131530131
person_id = 39061670

In [45]:
pt_df = data[data["PAT_ENC_CSN_ID"] == PAT_ENC_CSN_ID]

In [46]:
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(pt_df.dropna(axis=1))

,PAT_ENC_CSN_ID,person_id,visit_occurrence_id,age,EdDisposition_Admit,DepartmentName_BH EMERGENCY DEPARTMENT,Sex_Male,GenderIdentity_Male,FirstRace_White or Caucasian,Ethnicity_Non-Hispanic,PreferredLanguage_English,SmokingStatus_Never Smoker,AcuityLevel_Emergent,FinancialClass_BCBS,CC_ABDOMINAL PAIN,CIR007,ACE Inhibitors,Cephalosporin Antibiotics - 3rd Generation,ed_1744-2,ed_19123-9,ed_13536-8,ed_777-3,ed_785-6,ed_3094-0,ed_2951-2,ed_786-4,ed_789-8,ed_718-7,ed_2345-7,ed_787-2,ed_2823-3,ed_2075-0,ed_1962-0,ed_SpO2,ed_Temp,ed_Patient Acuity,ed_Pulse,ed_Pain Score,ed_Resp,ed_BP_Systolic,ed_BP_Diastolic,multilabel
35313,131530131,39061670,51144236,39.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,97.0,96.5,3.0,102.0,3.0,18.0,108.0,64.0,[Colitis]


In [48]:
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(labs[labs["person_id"] == person_id])
    display(vitals[vitals["person_id"] == person_id])
    display(diags[diags["PAT_ENC_CSN_ID"] == PAT_ENC_CSN_ID])
    display(meds[meds["person_id"] == person_id])
    display(pmh[pmh["person_id"] == person_id])
    display(demos[demos["PAT_ENC_CSN_ID"] == PAT_ENC_CSN_ID])

,visit_occurrence_id,person_id,EncounterEpicCsn,Name,Value,Comment,Abnormal,LoincName,LoincCode,ResultInstant,CollectionInstant
2794212,51144236,39061670,131530131,BUN,14,NaN,NaN,Urea nitrogen [Mass/volume] in Serum or Plasma,3094-0,2016-05-14 03:46:00.000,2016-05-14 03:15:00.000
2794213,51144236,39061670,131530131,Chloride,100,NaN,NaN,Chloride [Moles/volume] in Serum or Plasma,2075-0,2016-05-13 17:17:00.000,2016-05-13 16:50:00.000
2794214,51144236,39061670,131530131,Globulin,3.0,NaN,NaN,Globulin [Mass/volume] in Plasma,13536-8,2016-05-13 17:17:00.000,2016-05-13 16:50:00.000
2794215,51144236,39061670,131530131,Magnesium,1.6,NaN,NaN,Magnesium [Mass/volume] in Serum or Plasma,19123-9,2016-05-13 17:18:00.000,2016-05-13 16:50:00.000
3379311,51144236,39061670,131530131,Glucose,137,NaN,1.0,Glucose [Mass/volume] in Serum or Plasma,2345-7,2016-05-13 17:17:00.000,2016-05-13 16:50:00.000
3379312,51144236,39061670,131530131,Magnesium,1.3,NaN,1.0,Magnesium [Mass/volume] in Serum or Plasma,19123-9,2016-05-14 04:24:00.000,2016-05-14 03:15:00.000
3379313,51144236,39061670,131530131,MCHC,33.8,NaN,NaN,MCHC [Mass/volume] by Automated count,786-4,2016-05-13 17:00:00.000,2016-05-13 16:50:00.000
3379314,51144236,39061670,131530131,Sodium,142,NaN,NaN,Sodium [Moles/volume] in Serum or Plasma,2951-2,2016-05-13 17:17:00.000,2016-05-13 16:50:00.000
3668357,51144236,39061670,131530131,CO2,23,NaN,NaN,Bicarbonate [Moles/volume] in Plasma,1962-0,2016-05-14 03:46:00.000,2016-05-14 03:15:00.000
3668358,51144236,39061670,131530131,Hemoglobin,13.6,NaN,1.0,Hemoglobin [Mass/volume] in Blood,718-7,2016-05-14 03:25:00.000,2016-05-14 03:15:00.000


,visit_occurrence_id,person_id,EncounterEpicCsn,DisplayName,Value,TakenInstant
5754328,51144236,39061670,131530131,BP,70/40,2016-05-13 16:30:00.000
5754329,51144236,39061670,131530131,BP,93/52,2016-05-14 01:37:00.000
5754330,51144236,39061670,131530131,Pain Score,3,2016-05-13 16:11:00.000
5754331,51144236,39061670,131530131,Pulse,100,2016-05-14 04:57:00.000
5754332,51144236,39061670,131530131,Pulse,102,2016-05-14 05:01:00.000
5754333,51144236,39061670,131530131,Pulse,116,2016-05-13 20:41:00.000
5754334,51144236,39061670,131530131,Pulse,87,2016-05-13 16:49:00.000
5754335,51144236,39061670,131530131,Resp,18,2016-05-14 01:37:00.000
5754336,51144236,39061670,131530131,SpO2,97,2016-05-14 03:16:00.000
5754337,51144236,39061670,131530131,SpO2,98,2016-05-13 16:38:00.000


,visit_occurrence_id,person_id,PAT_ENC_CSN_ID,Name,Value
155117,51144236,39061670,131530131,Gastroenteritis,K52.9


,person_id,visit_occurrence_id,Name,PharmaceuticalClass,PharmaceuticalSubclass,StartInstant,EndInstant,NDC_code
10658063,39061670,51144236,LISINOPRIL 10 MG TABLET,"ANTIHYPERTENSIVES, ACE INHIBITORS",ACE Inhibitors,2016-08-08 00:00:00.000,2017-03-22 00:00:00.000,3700074136
12022327,39061670,51144236,LISINOPRIL 10 MG TABLET,"ANTIHYPERTENSIVES, ACE INHIBITORS",ACE Inhibitors,2016-01-11 00:00:00.000,2016-08-06 00:00:00.000,3700074136
14371459,39061670,51144236,LISINOPRIL 10 MG TABLET,"ANTIHYPERTENSIVES, ACE INHIBITORS",ACE Inhibitors,2015-07-16 00:00:00.000,2016-01-11 00:00:00.000,3700074136
22404620,39061670,51144236,LISINOPRIL 10 MG TABLET,"ANTIHYPERTENSIVES, ACE INHIBITORS",ACE Inhibitors,2012-12-23 00:00:00.000,2015-07-16 00:00:00.000,3700074136
30545254,39061670,51144236,LISINOPRIL 10 MG TABLET,"ANTIHYPERTENSIVES, ACE INHIBITORS",ACE Inhibitors,2017-03-22 00:00:00.000,2018-03-23 00:00:00.000,3700074136
44693868,39061670,51144236,CEFDINIR 300 MG CAPSULE,CEPHALOSPORIN ANTIBIOTICS - 3RD GENERATION,Cephalosporin Antibiotics - 3rd Generation,2015-09-28 00:00:00.000,2015-10-08 23:59:00.000,63370-163-15


,PAT_ID,person_id,Name,Value,DateValue
1491592,Z3826414,39061670,"Bipolar II disorder, most recent episode major...",F31.81,2019-06-03
1491593,Z3826414,39061670,"Depression, unspecified depression type",F32.9,2019-06-13
1491594,Z3826414,39061670,Essential hypertension,I10,2018-03-23
1491595,Z3826414,39061670,"Essential hypertension, benign",I10,2009-12-22
1491596,Z3826414,39061670,Family history of prostate cancer,Z80.42,2018-03-23
1491597,Z3826414,39061670,Gastroenteritis,K52.9,2016-05-14
1491598,Z3826414,39061670,Labile mood,R45.86,2019-06-03


,PAT_ENC_CSN_ID,PAT_ID,person_id,ArrivalInstant,DepartureInstant,EdDisposition,DepartmentName,visit_occurrence_id,age,Sex,GenderIdentity,FirstRace,Ethnicity,PreferredLanguage,SmokingStatus,AcuityLevel,FinancialClass,Name
124618,131530131,Z3826414,39061670,2016-05-13 16:05:00.000,2016-05-14 05:20:00.000,Admit,BH EMERGENCY DEPARTMENT,51144236,39.0,Male,Male,White or Caucasian,Non-Hispanic,English,Never Smoker,Emergent,BCBS,ABDOMINAL PAIN


In [ ]:
# described = data.describe()
# pd.set_option('display.float_format', lambda x: '%.3f' % x)
# described.sort_values("max", axis=1, ascending=False)